# Revista Contemporânea de Contabilidade – RCC

## Bibliotecas importadas

In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

## Funções básicas

In [2]:
def criar_query(key_word):
    return "+".join(key_word.split())

In [3]:
def listar_links_pagina(url_search):
    page = requests.get(url_search)
    soup = BeautifulSoup(page.content, 'html.parser')
    results = soup.find_all('h2', {'class': 'title'})
    a_links = []
    for item in results:
        try:
            remover_links = 'editorial' in item.find('a').text.lower()
            remover_links = remover_links or 'folha de rosto' in item.find('a').text.lower()
            remover_links = remover_links or 'dados da edição ' in item.find('a').text.lower()
            if not remover_links:
                a_links.append(item.find('a')['href'])
        except:
            pass
    return a_links

In [4]:
def listar_links(url_search_text, query, pg_num=1):
    a_links = [] 
    while True:
        url_search = url_search_text.replace('__query__', query).replace('__pg_num__', str(pg_num))
        tmp = listar_links_pagina(url_search)
        if len(tmp) != 0:
            pg_num += 1
            a_links += tmp
        else:
            break
    return a_links

In [5]:
url_search_text = 'https://www.revistas.usp.br/rco/search/search?query=__query__&searchJournal=45&authors=&title=&abstract=&galleyFullText=&discipline=&subject=&type=&coverage=&indexTerms=&dateFromMonth=&dateFromDay=&dateFromYear=2012&dateToMonth=&dateToDay=&dateToYear=2021&orderBy=score&orderDir=desc&searchPage=__pg_num__#results'

links = listar_links(url_search_text, 'contabilidade+ambiental', pg_num=1)

link = links[1]

link


'https://www.revistas.usp.br/rco/article/view/56693'

In [6]:
links

['https://www.revistas.usp.br/rco/article/view/59345',
 'https://www.revistas.usp.br/rco/article/view/56693',
 'https://www.revistas.usp.br/rco/article/view/56666',
 'https://www.revistas.usp.br/rco/article/view/82709',
 'https://www.revistas.usp.br/rco/article/view/56665',
 'https://www.revistas.usp.br/rco/article/view/56667',
 'https://www.revistas.usp.br/rco/article/view/55448',
 'https://www.revistas.usp.br/rco/article/view/56668',
 'https://www.revistas.usp.br/rco/article/view/161973',
 'https://www.revistas.usp.br/rco/article/view/55456',
 'https://www.revistas.usp.br/rco/article/view/45399',
 'https://www.revistas.usp.br/rco/article/view/55586',
 'https://www.revistas.usp.br/rco/article/view/97726',
 'https://www.revistas.usp.br/rco/article/view/56691',
 'https://www.revistas.usp.br/rco/article/view/144885',
 'https://www.revistas.usp.br/rco/article/view/52657',
 'https://www.revistas.usp.br/rco/article/view/56690',
 'https://www.revistas.usp.br/rco/article/view/56695',
 'https:

In [7]:
page = requests.get(link)
soup = BeautifulSoup(page.content, 'html.parser')

In [8]:
def completar_lista(lista):
    if len(lista) < 6:
        lista += (6 - len(lista)) * ['']
    return lista

In [9]:
def limpar_afiliacao(afiliacao):
    afiliacao = ' '.join(afiliacao.split())
    afiliacao = afiliacao.split(',')[0]
    afiliacao = afiliacao.split(' - ')[0]
    afiliacao = afiliacao.split('(')[0]
    afiliacao = afiliacao.split('/')[0]
    return afiliacao.strip()

In [10]:
def captura_titulo(soup):
    try:
        title = soup.find('h1', {'class': 'page_title'}).text
        title = ' '.join(title.split())
    except:
        title = ''
    return {'Título': title}

In [11]:
captura_titulo(soup)

{'Título': 'AVALIAÇÃO DA INFLUÊNCIA DOS STAKEHOLDERS NA PROATIVIDADE AMBIENTAL DE EMPRESAS BRASILEIRAS'}

In [12]:
def captura_data(soup):
    try:
        date_tag = soup.find('div', {'class': 'item published'})
        date = date_tag.find('div', {'class': 'value'}).text
        date = ' '.join(date.split())
    except:
        date = ''
    return {'Data de Publicação': date[:10]}

In [13]:
captura_data(soup)

{'Data de Publicação': '2013-07-10'}

In [14]:
def captura_resumo(soup):
    try:
        abstract_section = soup.find('section', {'class': 'item abstract'})
        abstract = abstract_section.text
        abstract = abstract.replace('Resumo:', '')
        abstract = abstract.replace('Resumo', '').strip()
        #abstract = ' '.join(abstract.split())
    except:
        abstract = ''
    return {'Resumo': abstract}

In [15]:
captura_resumo(soup)

{'Resumo': 'O artigo analisa a influência dos stakeholders na proatividade ambiental de empresas brasileiras. A pesquisa foi realizada com 112 empresas brasileiras com o objetivo de testar a hipótese de que a pressão por parte dos stakeholders influencia positivamente as atividades de gestão ambiental das empresas brasileiras. A análise fatorial agrupou os stakeholders nas categorias “mercado” e “não mercado”. Os primeiros participam diretamente da cadeia de suprimento e incluem fornecedores, clientes e concorrentes internacionais e domésticos, empregados, subcontratados e sindicatos. Os stakeholders “não mercado”, por sua vez, não participam diretamente da cadeia de suprimentos e são caracterizados pelos acionistas, governo, mídia e ONGs. Os resultados dos modelos econométricos demonstraram que os stakeholders possuem um efeito significativo e positivo sobre as ações de proatividade ambiental, envolvendo práticas de planejamento, operações e comunicação. Essa pressão é mais efetiva qu

In [16]:
def captura_palavras_chave(soup):
    try:
        kw_section = soup.find('section', {'class': 'item keywords'})
        kw = kw_section.text.replace('Palavras-chave:', '')
        kw = ' '.join(kw.split())
        kw = kw.replace(',', '.').replace(';', '.')
    except:
        kw = ''
    if kw == '':
        try:
            kw_section = soup.find('section', {'class': 'item abstract'})
            p_list = kw_section.find_all('p')
            for p in p_list:
                if 'Palavras-chave' in p.text:
                    kw_full = p.text
                    kw = kw_full.replace('Palavras-chave', '').replace(':', '')
                    kw = ' '.join(kw.split())
                    kw = kw.replace(',', '.').replace(';', '.')
        except:
            pass
    return {'Palavras-chave': kw}

In [17]:
captura_palavras_chave(soup)

{'Palavras-chave': 'environmental management. environmental pressure. environmental proactivity. strategy. stakeholders. sustainability.'}

In [18]:
def captura_autores(soup):
    autores = []
    try:
        authors_ul = soup.find('ul', {'class': 'authors'})
        autores = authors_ul.find_all('span', {'class': 'name'})
        autores = [' '.join(a.text.split()) for a in autores]
    except:
        pass
    qtd_autores = len(autores)
    autores = completar_lista(autores)
    autores = [qtd_autores] + autores
    
    key_autores = [f"Autor {i}" for i in range(1, 7)]
    key_autores = ['Qtd. de Autores'] + key_autores

    return dict(zip(key_autores, autores))

In [19]:
captura_autores(soup)

{'Qtd. de Autores': 3,
 'Autor 1': 'Mônica Cavalcanti Sá Abreu',
 'Autor 2': 'Francisco Cesar Castro',
 'Autor 3': 'José Carlos Lazaro',
 'Autor 4': '',
 'Autor 5': '',
 'Autor 6': ''}

In [20]:
def captura_afiliacao(soup):
    afiliacao = []
    try:
        authors_ul = soup.find('ul', {'class': 'authors'})
        afiliacao = authors_ul.find_all('span', {'class': 'affiliation'})
        afiliacao = [' '.join(a.text.split()) for a in afiliacao]
        afiliacao = [limpar_afiliacao(a) for a in afiliacao]
    except:
        pass
    afiliacao = completar_lista(afiliacao)
    
    key_affiliation = [f"Afiliação {i}" for i in range(1, 7)]

    return dict(zip(key_affiliation, afiliacao))

In [21]:
captura_afiliacao(soup)

{'Afiliação 1': 'Universidade Federal do Ceará',
 'Afiliação 2': 'Faculdade Metropolitana de Fortaleza',
 'Afiliação 3': 'Universidade Federal do Ceará',
 'Afiliação 4': '',
 'Afiliação 5': '',
 'Afiliação 6': ''}

In [22]:
def realizar_consulta(key_words_list):
    periodico_nome_dict = {'Revista': 'Revista de Contabilidade e Organizações – RCO'}
    url_search_text = 'https://www.revistas.usp.br/rco/search/search?query=__query__&searchJournal=45&authors=&title=&abstract=&galleyFullText=&discipline=&subject=&type=&coverage=&indexTerms=&dateFromMonth=&dateFromDay=&dateFromYear=2012&dateToMonth=&dateToDay=&dateToYear=2021&orderBy=score&orderDir=desc&searchPage=__pg_num__#results'
    a_links = []
    for key_word in key_words_list:
        query = criar_query(key_word)
        a_links += listar_links(url_search_text, query)
        
    resultados = []
    for link in a_links:
        link_dict = {'Url Artigo': link}
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        dict_artigo = {
            **captura_data(soup),
            **captura_titulo(soup),
            **captura_resumo(soup),
            **captura_palavras_chave(soup),
            **captura_autores(soup),
            **captura_afiliacao(soup),
            **link_dict,
            **periodico_nome_dict
        }
        resultados.append(dict_artigo)
    df = pd.DataFrame(resultados)
    df['Data de Publicação'] = pd.to_datetime(df['Data de Publicação'], format='%Y-%m-%d')
    return df

## Pesquisa por artigos

In [23]:
key_words_list = ['contabilidade ambiental', 'balanço social', 'relato integrado', 'nbct 15']
df = realizar_consulta(key_words_list)
df.head(3)

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afiliação 1,Afiliação 2,Afiliação 3,Afiliação 4,Afiliação 5,Afiliação 6,Url Artigo,Revista
0,2013-07-30,PESQUISA EM CONTABILIDADE SOCIAL COMO SE O MUN...,O objetivo deste ensaio é oferecer uma autocrí...,,3,Rob Gray,Jesse Dillard,Crawford Spence,,,,,,,,,,https://www.revistas.usp.br/rco/article/view/5...,Revista de Contabilidade e Organizações – RCO
1,2013-07-10,AVALIAÇÃO DA INFLUÊNCIA DOS STAKEHOLDERS NA PR...,O artigo analisa a influência dos stakeholders...,environmental management. environmental pressu...,3,Mônica Cavalcanti Sá Abreu,Francisco Cesar Castro,José Carlos Lazaro,,,,Universidade Federal do Ceará,Faculdade Metropolitana de Fortaleza,Universidade Federal do Ceará,,,,https://www.revistas.usp.br/rco/article/view/5...,Revista de Contabilidade e Organizações – RCO
2,2013-07-10,EVIDENCIAÇÃO DE INFORMAÇÕES AMBIENTAIS: PROPOS...,O artigo tem por objetivo elencar atributos am...,Attributes. Environmental Disclosure. Disclosu...,3,Ramon K. B. Bachmann,Leandro M. Carneiro,Márcia M. S. B. Espejo,,,,Universidade Federal do Paraná,Universidade Federal do Paraná,Universidade Federal do Paraná,,,,https://www.revistas.usp.br/rco/article/view/5...,Revista de Contabilidade e Organizações – RCO


In [24]:
df.to_csv('../data/RCO_FEA_USP.csv', index=False)

In [25]:
df

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afiliação 1,Afiliação 2,Afiliação 3,Afiliação 4,Afiliação 5,Afiliação 6,Url Artigo,Revista
0,2013-07-30,PESQUISA EM CONTABILIDADE SOCIAL COMO SE O MUN...,O objetivo deste ensaio é oferecer uma autocrí...,,3,Rob Gray,Jesse Dillard,Crawford Spence,,,,,,,,,,https://www.revistas.usp.br/rco/article/view/5...,Revista de Contabilidade e Organizações – RCO
1,2013-07-10,AVALIAÇÃO DA INFLUÊNCIA DOS STAKEHOLDERS NA PR...,O artigo analisa a influência dos stakeholders...,environmental management. environmental pressu...,3,Mônica Cavalcanti Sá Abreu,Francisco Cesar Castro,José Carlos Lazaro,,,,Universidade Federal do Ceará,Faculdade Metropolitana de Fortaleza,Universidade Federal do Ceará,,,,https://www.revistas.usp.br/rco/article/view/5...,Revista de Contabilidade e Organizações – RCO
2,2013-07-10,EVIDENCIAÇÃO DE INFORMAÇÕES AMBIENTAIS: PROPOS...,O artigo tem por objetivo elencar atributos am...,Attributes. Environmental Disclosure. Disclosu...,3,Ramon K. B. Bachmann,Leandro M. Carneiro,Márcia M. S. B. Espejo,,,,Universidade Federal do Paraná,Universidade Federal do Paraná,Universidade Federal do Paraná,,,,https://www.revistas.usp.br/rco/article/view/5...,Revista de Contabilidade e Organizações – RCO
3,2015-12-31,ESTRATÉGIAS COMPETITIVAS E DESEMPENHO ECONÔMIC...,Este estudo teve como objetivo analisar a infl...,Estratégias Competitivas. Tipologias de Miles ...,4,Caroline Sulzbach Pletsch,Cristian Bau Dal Magro,Alini da Silva,Carlos Eduardo Facin Lavarda,,,Universidade Regional de Blumenau-Furb,Universidade Regional de Blumenau,Universidade Regional de Blumenau,Universidade Regional de Blumenau,,,https://www.revistas.usp.br/rco/article/view/8...,Revista de Contabilidade e Organizações – RCO
4,2013-07-10,PERCEPÇÕES DE GESTORES SOBRE AS CONTRIBUIÇÕES ...,A necessidade de harmonizar o desenvolvimento ...,Environmental accounting practices. ISO 14001 ...,4,Anisio C. Pereira,Silvia B. de Melo,Vilma G. Slomski,Elionor F. J. Weffort,,,Fundação Escola de Comércio Álvares Penteado,Faculdades Integradas de Fernandópolis,Fundação Escola de Comércio Álvares Penteado,Centro Universitário Álvares Penteado,,,https://www.revistas.usp.br/rco/article/view/5...,Revista de Contabilidade e Organizações – RCO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,2015-09-29,DO TEXTO AO CONTEXTO: O USO DA CONTABILIDADE G...,O presente artigo tem por objetivo analisar a ...,Contabilidade Gerencial. Comunicação. Bakhtin....,2,Patricia Villa Costa Vaz,Márcia Maria dos Santos Bortolocci Espejo,,,,,Universidade Federal do Paraná,Universidade Federal do Paraná,,,,,https://www.revistas.usp.br/rco/article/view/6...,Revista de Contabilidade e Organizações – RCO
65,2014-12-18,VARIABILIDADE DOS ÍNDICES DE ENDIVIDAMENTO EM ...,O estudo objetiva analisar se há variabilidade...,Indebtedness ratios. Contingent liabilities. I...,3,Jorge Eduardo Scarpin,Edson Roberto Macohon,Lara Fabiana Dallabona,,,,Universidade Federal do Paraná,Universidade Estadual do Centro-Oeste,Universidade Regional de Blumenau,,,,https://www.revistas.usp.br/rco/article/view/6...,Revista de Contabilidade e Organizações – RCO
66,2020-05-28,Terceirização de portais de transparência fisc...,A terceirização de portais de transparência po...,Descontinuidade de Portais de Transparência. G...,4,Lidiane Nazaré da Silva Dias,André Carlos Busanelli de Aquino,Polyana Batista da Silva,Francivaldo dos Santos Albuquerque,,,Universidade Federal do Pará,Universidade de São Paulo,Universidade de São Paulo,Universidade Federal do Pará,,,https://www.revistas.usp.br/rco/article/view/1...,Revista de Contabilidade e Organizações – RCO
67,2014-08-18,PRÁTICAS DE GESTÃO DE CUSTOS LOGÍSTICOS: ESTUD...,O objetivo do artigo é identificar as práticas...,Gestão Estratégica de Custos. Cadeia de Suprim...,3,Marcos Antonio Souza,Cristiano Rempel,Jorge Luiz Rosa Silva,,,,UNISINOS,Mestrando em Ciência